In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load dataset
df = pd.read_csv('diabetes_dataset.csv')

# Feature selection
features = ['Glucose', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
X = df[features].values
y = df['Outcome'].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

def triangular_mf(x, a, b, c):
    return max(min((x - a) / (b - a), (c - x) / (c - b)), 0)

def evaluate_rules(features):
    Glucose, INS, BMI, DPF, Age = features
    
    # Membership functions for each feature
    Glucoselow = triangular_mf(Glucose, 0, 85, 100)
    Glucosemedium = triangular_mf(Glucose, 85, 125, 160)
    Glucosehigh = triangular_mf(Glucose, 125, 180, 200)
    
    INSlow = triangular_mf(INS, 0, 100, 200)
    INSmedium = triangular_mf(INS, 100, 200, 400)
    INShigh = triangular_mf(INS, 200, 400, 600)
    
    BMIlow = triangular_mf(BMI, 0, 20, 30)
    BMImedium = triangular_mf(BMI, 20, 30, 40)
    BMIhigh = triangular_mf(BMI, 30, 40, 50)
    
    DPFlow = triangular_mf(DPF, 0, 0.3, 0.6)
    DPFmedium = triangular_mf(DPF, 0.3, 0.6, 1.0)
    DPFhigh = triangular_mf(DPF, 0.6, 1.0, 2.0)
    
    Ageyoung = triangular_mf(Age, 0, 30, 40)
    
    # Fuzzy rules
    DMverylow = max(Glucoselow, INSlow, BMIlow, DPFlow, Ageyoung)
    DMlow = max(Glucoselow, INSlow, BMIhigh, DPFlow, Ageyoung)
    DMmedium = max(Glucosemedium, INShigh, BMIhigh, DPFmedium, Ageyoung)
    DMhigh = max(Glucosehigh, INSmedium, BMIhigh, DPFhigh, Ageyoung)
    
    # Aggregate into a single value
    # For simplicity, taking the maximum of the fuzzy DM values
    DM = max(DMverylow, DMlow, DMmedium, DMhigh)
    
    return DM

# Apply rules to compute fuzzy DM value
fuzzy_results_train = [evaluate_rules(features) for features in X_train]
fuzzy_results_test = [evaluate_rules(features) for features in X_test]

# Defuzzification: For simplicity, treating the fuzzy DM value as a probability and rounding to get a binary prediction
y_pred_train = [round(result) for result in fuzzy_results_train]
y_pred_test = [round(result) for result in fuzzy_results_test]

# Compute accuracy
train_accuracy = accuracy_score(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test, y_pred_test)

print(f"Train Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")


Train Accuracy: 0.36156351791530944
Test Accuracy: 0.2987012987012987


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# Load the dataset
dataset = pd.read_csv('diabetes_dataset.csv')
X = dataset.drop('Outcome', axis=1).values
y = dataset['Outcome'].values

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define triangular membership function
def triangular_mf(x, a, b, c):
    return max(min((x - a) / (b - a), (c - x) / (c - b)), 0)

# Fuzzy rule evaluation
def evaluate_rules(features):
    Glucose, INS, BMI, DPF, Age = features
    
    # Membership functions for each feature
    Glucoselow = triangular_mf(Glucose, 0, 85, 100)
    Glucosemedium = triangular_mf(Glucose, 85, 125, 160)
    Glucosehigh = triangular_mf(Glucose, 125, 180, 200)
    
    INSlow = triangular_mf(INS, 0, 100, 200)
    INSmedium = triangular_mf(INS, 100, 200, 400)
    INShigh = triangular_mf(INS, 200, 400, 600)
    
    BMIlow = triangular_mf(BMI, 0, 20, 30)
    BMImedium = triangular_mf(BMI, 20, 30, 40)
    BMIhigh = triangular_mf(BMI, 30, 40, 50)
    
    DPFlow = triangular_mf(DPF, 0, 0.3, 0.6)
    DPFmedium = triangular_mf(DPF, 0.3, 0.6, 1.0)
    DPFhigh = triangular_mf(DPF, 0.6, 1.0, 2.0)
    
    Ageyoung = triangular_mf(Age, 0, 30, 40)
    
    # Fuzzy rules
    DMverylow = max(Glucoselow, INSlow, BMIlow, DPFlow, Ageyoung)
    DMlow = max(Glucoselow, INSlow, BMIhigh, DPFlow, Ageyoung)
    DMmedium = max(Glucosemedium, INShigh, BMIhigh, DPFmedium, Ageyoung)
    DMhigh = max(Glucosehigh, INSmedium, BMIhigh, DPFhigh, Ageyoung)
    
    # Aggregate into a single value
    # For simplicity, taking the maximum of the fuzzy DM values
    DM = max(DMverylow, DMlow, DMmedium, DMhigh)
    
    return DM

# Define fitness function
def fitness(params):
    fuzzy_results_train = [evaluate_rules(features, params) for features in X_train]
    y_pred_train = [round(result) for result in fuzzy_results_train]
    
    acc = accuracy_score(y_train, y_pred_train)
    
    tn, fp, fn, tp = confusion_matrix(y_train, y_pred_train).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    # Combine the metrics
    return 1 - (acc * 0.33 + sensitivity * 0.33 + specificity * 0.34)

# Harris Hawk Optimization
class HarrisHawkOptimization:
    def __init__(self, fitness_func, dim, n_hawks=30, n_iterations=100):
        self.fitness_func = fitness_func
        self.dim = dim
        self.n_hawks = n_hawks
        self.n_iterations = n_iterations
        self.hawks = np.random.uniform(0, 100, (self.n_hawks, self.dim))
    
    def optimize(self):
        for iteration in range(self.n_iterations):
            for i in range(self.n_hawks):
                self._update_position(i)
                
    def _update_position(self, i):
        hawk1, hawk2 = self.hawks[np.random.choice(range(self.n_hawks), 2, replace=False)]
        
        if self.fitness_func(self.hawks[i]) > self.fitness_func(hawk1):
            self.hawks[i] = (self.hawks[i] + hawk1) / 2
        else:
            self.hawks[i] = (self.hawks[i] + hawk2) / 2

# Initialize HHO
hho = HarrisHawkOptimization(fitness, dim=15, n_hawks=30, n_iterations=100)

# Run Optimization
hho.optimize()

# Get the best hawk (solution)
best_hawk = min(hho.hawks, key=fitness)
print("Best Parameters: ", best_hawk)

# Test the classifier on the test set
fuzzy_results_test = [evaluate_rules(features, best_hawk) for features in X_test]
y_pred_test = [round(result) for result in fuzzy_results_test]

# Calculate performance metrics
acc = accuracy_score(y_test, y_pred_test)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_test).ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print(f"Accuracy: {acc}")
print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")


In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# Load the dataset
dataset = pd.read_csv('diabetes_dataset.csv')
X = dataset[['Glucose', 'INS', 'BMI', 'DPF', 'Age']].values
y = dataset['Outcome'].values


# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define triangular membership function
def triangular_mf(x, a, b, c):
    return max(min((x - a) / (b - a), (c - x) / (c - b)), 0)

# Fuzzy rule evaluation
def evaluate_rules(features, params):
    # Assuming params will be used for some purpose, though it's not used in your original code.
    Glucose, INS, BMI, DPF, Age = features
    
    # Membership functions for each feature
    Ageyoung = triangular_mf(Age, 0, 30, 40)
    
    Glucoselow = triangular_mf(Glucose, 0, 85, 100)
    Glucosemedium = triangular_mf(Glucose, 85, 125, 160)
    Glucosehigh = triangular_mf(Glucose, 125, 180, 200)
    
    INSlow = triangular_mf(INS, 0, 100, 200)
    INSmedium = triangular_mf(INS, 100, 200, 400)
    INShigh = triangular_mf(INS, 200, 400, 600)
    
    BMIlow = triangular_mf(BMI, 0, 20, 30)
    BMImedium = triangular_mf(BMI, 20, 30, 40)
    BMIhigh = triangular_mf(BMI, 30, 40, 50)
    
    DPFlow = triangular_mf(DPF, 0, 0.3, 0.6)
    DPFmedium = triangular_mf(DPF, 0.3, 0.6, 1.0)
    DPFhigh = triangular_mf(DPF, 0.6, 1.0, 2.0)
    
    
    # Fuzzy rules
    DMverylow1 = max(Glucoselow, INSlow, BMIlow, DPFlow, Ageyoung)
    DMlow = max(Glucoselow, INSlow, BMIhigh, DPFlow, Ageyoung)
    DMmedium = max(Glucosemedium, INShigh, BMIhigh, DPFmedium, Ageyoung)
    DMhigh = max(Glucosehigh, INSmedium, BMIhigh, DPFhigh, Ageyoung)
    DMverylow2 = max(Glucoselow, INSlow, BMImedium, DPFlow, Ageyoung)

    # Aggregate into a single value
    DM = max(DMverylow1, DMlow, DMmedium, DMhigh, DMverylow2)
    
    return DM

# Define fitness function
def fitness(params):
    fuzzy_results_train = [evaluate_rules(features, params) for features in X_train]
    y_pred_train = [round(result) for result in fuzzy_results_train]
    
    acc = accuracy_score(y_train, y_pred_train)
    tn, fp, fn, tp = confusion_matrix(y_train, y_pred_train).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    return 1 - (acc * 0.33 + sensitivity * 0.33 + specificity * 0.34)

# Harris Hawk Optimization
class HarrisHawkOptimization:
    def __init__(self, fitness_func, dim, n_hawks=30, n_iterations=100):
        self.fitness_func = fitness_func
        self.dim = dim
        self.n_hawks = n_hawks
        self.n_iterations = n_iterations
        self.hawks = np.random.uniform(0, 100, (self.n_hawks, self.dim))
        self.rabbit = None  # Best solution

    def _find_rabbit(self):
        self.rabbit = min(self.hawks, key=self.fitness_func)

    def _update_position(self, hawk):
        E1 = np.random.uniform(0, 1)  # Exploration factor
        E2 = np.random.uniform(0, 1)  # Exploitation factor

        if E1 < 0.5:  # Exploitation phase
            if E2 < 0.5:
                hawk = self.rabbit - np.random.uniform(0, 1) * abs(self.rabbit - hawk)
            else:
                hawk = self.rabbit + np.random.uniform(0, 1) * abs(self.rabbit - hawk)
        else:  # Exploration phase
            hawk = np.random.uniform(0, 100, self.dim)
        
        return hawk

    def optimize(self):
        for iteration in range(self.n_iterations):
            self._find_rabbit()

            for i in range(self.n_hawks):
                new_hawk = self._update_position(self.hawks[i])

                if self.fitness_func(new_hawk) < self.fitness_func(self.hawks[i]):
                    self.hawks[i] = new_hawk

# Initialize HHO
hho = HarrisHawkOptimization(fitness, dim=5, n_hawks=30, n_iterations=100)

# Run Optimization
hho.optimize()

# Get the best hawk (solution)
best_hawk = min(hho.hawks, key=fitness)
print("Best Parameters: ", best_hawk)

# Test the classifier on the test set
fuzzy_results_test = [evaluate_rules(features, best_hawk) for features in X_test]
y_pred_test = [round(result) for result in fuzzy_results_test]

# Calculate performance metrics
acc = accuracy_score(y_test, y_pred_test)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_test).ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print(f"Accuracy: {acc}")
print(f"Sensitivity: {sensitivity}")
print(f"Specificity: {specificity}")


Best Parameters:  [71.59840833 48.42757218 63.41311438 65.22931935 63.48868432]
Accuracy: 0.35714285714285715
Sensitivity: 1.0
Specificity: 0.0


In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# Load the dataset
dataset = pd.read_csv('diabetes_dataset.csv')
X = dataset[['Glucose', 'INS', 'BMI', 'DPF', 'Age']].values
y = dataset['Outcome'].values

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def triangular_mf(x, C, L, R):
    return max(min((x - L) / (C - L), (R - x) / (R - C)), 0)

def evaluate_rules(features, params):
    Glucose, INS, BMI, DPF, Age = features
    
    # Glucose
    k, w = params[0], params[1]
    C, L, R = 85, 0, 100
    Glucoselow = triangular_mf(Glucose, C + k - w, L + k - w, R + k - w)
    
    # INS
    k, w = params[2], params[3]
    C, L, R = 100, 0, 200
    INSlow = triangular_mf(INS, C + k - w, L + k - w, R + k - w)
    
    # BMI
    k, w = params[4], params[5]
    C, L, R = 20, 0, 30
    BMIlow = triangular_mf(BMI, C + k - w, L + k - w, R + k - w)
    
    # DPF
    k, w = params[6], params[7]
    C, L, R = 0.3, 0, 0.6
    DPFlow = triangular_mf(DPF, C + k - w, L + k - w, R + k - w)
    
    # Age
    k, w = params[8], params[9]
    C, L, R = 30, 0, 40
    Ageyoung = triangular_mf(Age, C + k - w, L + k - w, R + k - w)
    
    # Fuzzy rules
    DM = max(Glucoselow, INSlow, BMIlow, DPFlow, Ageyoung)
    
    return DM

def fitness(params):
    fuzzy_results_train = [evaluate_rules(features, params) for features in X_train]
    y_pred_train = [round(result) for result in fuzzy_results_train]
    
    acc = accuracy_score(y_train, y_pred_train)
    tn, fp, fn, tp = confusion_matrix(y_train, y_pred_train).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    return 1 - (acc * 0.33 + sensitivity * 0.33 + specificity * 0.34)

class HarrisHawkOptimization:
    def __init__(self, fitness_func, dim, n_hawks=30, n_iterations=100):
        self.fitness_func = fitness_func
        self.dim = dim
        self.n_hawks = n_hawks
        self.n_iterations = n_iterations
        self.hawks = np.random.uniform(0, 100, (self.n_hawks, self.dim))
        self.rabbit = None

    def _find_rabbit(self):
        self.rabbit = min(self.hawks, key=self.fitness_func)

    def _update_position(self, hawk):
        E1 = np.random.uniform(0, 1)
        E2 = np.random.uniform(0, 1)
        
        if E1 < 0.5:
            if E2 < 0.5:
                hawk = self.rabbit - np.random.uniform(0, 1) * abs(self.rabbit - hawk)
            else:
                hawk = self.rabbit + np.random.uniform(0, 1) * abs(self.rabbit - hawk)
        else:
            hawk = np.random.uniform(0, 100, self.dim)
        
        return hawk

    def optimize(self):
        for iteration in range(self.n_iterations):
            self._find_rabbit()
            for i in range(self.n_hawks):
                new_hawk = self._update_position(self.hawks[i])
                if self.fitness_func(new_hawk) < self.fitness_func(self.hawks[i]):
                    self.hawks[i] = new_hawk

# Initialize HHO
hho = HarrisHawkOptimization(fitness, dim=10, n_hawks=30, n_iterations=100)

# Run Optimization
hho.optimize()

# Get the best hawk
best_hawk = min(hho.hawks, key=fitness)
print("Best Parameters: ", best_hawk)

# Evaluate on test set
fuzzy_results_test = [evaluate_rules(features, best_hawk) for features in X_test]
y_pred_test = [round(result) for result in fuzzy_results_test]

# Calculate metrics
acc = accuracy_score(y_test, y_pred_test)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_test).ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print(f"Accuracy based on optimized parameters: {acc}")
print(f"Sensitivity based on optimized parameters: {sensitivity}")
print(f"Specificity based on optimized parameters: {specificity}")


Best Parameters:  [162.84207893  51.00093045 100.41541327  27.94006881  61.30857807
 140.71789891 113.71959554  56.14423975  61.42708761  34.41422777]
Accuracy based on optimized parameters: 0.6363636363636364
Sensitivity based on optimized parameters: 0.6
Specificity based on optimized parameters: 0.6565656565656566


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# Load the dataset
dataset = pd.read_csv('diabetes_dataset.csv')
X = dataset[['Glucose', 'INS', 'BMI', 'DPF', 'Age']].values
y = dataset['Outcome'].values

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define triangular membership function with optimized parameters
def triangular_mf(x, a, b, c, ki, wi):
    a = (a + ki) - wi
    b = (b + ki) - wi
    c = (c + ki) - wi
    return max(min((x - a) / (b - a), (c - x) / (c - b)), 0)

def fitness(params):
    fuzzy_results_train = [evaluate_rules(features, params) for features in X_train]
    y_pred_train = [round(result) for result in fuzzy_results_train]
    
    acc = accuracy_score(y_train, y_pred_train)
    tn, fp, fn, tp = confusion_matrix(y_train, y_pred_train).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    return 1 - (acc * 0.33 + sensitivity * 0.33 + specificity * 0.34)

class HarrisHawkOptimization:
    def __init__(self, fitness_func, dim, n_hawks=30, n_iterations=100):
        self.fitness_func = fitness_func
        self.dim = dim
        self.n_hawks = n_hawks
        self.n_iterations = n_iterations
        self.hawks = np.random.uniform(0, 100, (self.n_hawks, self.dim))
        self.rabbit = None

    def _find_rabbit(self):
        self.rabbit = min(self.hawks, key=self.fitness_func)

    def _update_position(self, hawk):
        E1 = np.random.uniform(0, 1)
        E2 = np.random.uniform(0, 1)
        
        if E1 < 0.5:
            if E2 < 0.5:
                hawk = self.rabbit - np.random.uniform(0, 1) * abs(self.rabbit - hawk)
            else:
                hawk = self.rabbit + np.random.uniform(0, 1) * abs(self.rabbit - hawk)
        else:
            hawk = np.random.uniform(0, 100, self.dim)
        
        return hawk

    def optimize(self):
        for iteration in range(self.n_iterations):
            self._find_rabbit()
            for i in range(self.n_hawks):
                new_hawk = self._update_position(self.hawks[i])
                if self.fitness_func(new_hawk) < self.fitness_func(self.hawks[i]):
                    self.hawks[i] = new_hawk

# Fuzzy rule evaluation with optimized parameters
def evaluate_rules(features, params):
    Glucose, INS, BMI, DPF, Age = features
    ki, wi = params  # or ki = params[0], wi = params[1] if params is an array
    
    # Membership functions for each feature, adjusted based on ki and wi
    Ageyoung = triangular_mf(Age, 0, 30, 40, ki, wi)
    
    Glucoselow = triangular_mf(Glucose, 0, 85, 100, ki, wi)
    Glucosemedium = triangular_mf(Glucose, 85, 125, 160, ki, wi)
    Glucosehigh = triangular_mf(Glucose, 125, 180, 200, ki, wi)
    
    INSlow = triangular_mf(INS, 0, 100, 200, ki, wi)
    INSmedium = triangular_mf(INS, 100, 200, 400, ki, wi)
    INShigh = triangular_mf(INS, 200, 400, 600, ki, wi)
    
    BMIlow = triangular_mf(BMI, 0, 20, 30, ki, wi)
    BMImedium = triangular_mf(BMI, 20, 30, 40, ki, wi)
    BMIhigh = triangular_mf(BMI, 30, 40, 50, ki, wi)
    
    DPFlow = triangular_mf(DPF, 0, 0.3, 0.6, ki, wi)
    DPFmedium = triangular_mf(DPF, 0.3, 0.6, 1.0, ki, wi)
    DPFhigh = triangular_mf(DPF, 0.6, 1.0, 2.0, ki, wi)
    
    # Fuzzy rules
    DMverylow1 = max(Glucoselow, INSlow, BMIlow, DPFlow, Ageyoung)
    DMlow = max(Glucoselow, INSlow, BMIhigh, DPFlow, Ageyoung)
    DMmedium = max(Glucosemedium, INShigh, BMIhigh, DPFmedium, Ageyoung)
    DMhigh = max(Glucosehigh, INSmedium, BMIhigh, DPFhigh, Ageyoung)
    DMverylow2 = max(Glucoselow, INSlow, BMImedium, DPFlow, Ageyoung)

    # Aggregate into a single value for DM
    DM = max(DMverylow1, DMlow, DMmedium, DMhigh, DMverylow2)
    
    return DM

# Initialize HHO
hho = HarrisHawkOptimization(fitness, dim=2, n_hawks=30, n_iterations=100)  # Here dim=2 for ki and wi

# Run Optimization
hho.optimize()

# Get the best hawk (solution)
best_hawk = min(hho.hawks, key=fitness)
print("Best Parameters ki and wi: ", best_hawk)

# Test the classifier on the test set using optimized parameters
fuzzy_results_test = [evaluate_rules(features, best_hawk) for features in X_test]
y_pred_test = [round(result) for result in fuzzy_results_test]

# Calculate performance metrics
acc = accuracy_score(y_test, y_pred_test)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_test).ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print(f"Accuracy based on optimized parameters: {acc}")
print(f"Sensitivity based on optimized parameters: {sensitivity}")
print(f"Specificity based on optimized parameters: {specificity}")
